binanry decisionn variable sp (eg berkely admissions)
2 grouping varibles & a binary
 - 1 grouping variable eg gender
 - grouping variable has class imbalance
mixed: use continus variables to define the decsion variable.


## To induce SP in rates

we need the grouping variable to have onen rate of success in decision variable that is the opposite rate of the overall, so we need to sam

In [2]:
import numpy as np
import pandas as pd
import string
import matplotlib.pylab as plt

In [131]:
p_dept = [.15,.15,.1,.6]
p_gender_dept = [[.7, .3],[.8,.2],[.85,.15],[.2,.8]]
gender_list = ['F','M']
p_race_dept = [[.8,.1,.1], [.7,.13,.17],[.5,.2,.3],[.85,.07,.08]]
race_list = ['W','B','H']
p_admit_dept = [[.15,.85], [.18,.82], [.25,.75],[.3,.7]]
# need to have higher accept in the larger subgroup, larger subgroup should have opposite protected class balance
N = 1000

d = np.random.choice(list(range(len(p_dept))), size=N, p =p_dept)
g = [np.random.choice(gender_list, p =p_gender_dept[d_i]) for d_i in d]
a = [np.random.choice([1,0],p = p_admit_dept[d_i]) for d_i in d]
r = [np.random.choice(race_list, p =p_race_dept[d_i]) for d_i in d]
data = [[d_i,g_i,a_i,r_i] for d_i, g_i,a_i,r_i in zip(d,g,a,r)]

df = pd.DataFrame(data = data, columns=['department','gender','decision','race'])
df.head()

,department,gender,decision,race
0,3,F,1,H
1,3,M,0,W
2,3,M,0,B
3,0,F,0,W
4,0,F,0,W


In [132]:
df.groupby('gender')['decision'].mean()

gender
F    0.203271
M    0.284965
Name: decision, dtype: float64

In [133]:
df.groupby(['gender','department']).mean().unstack()

decision                              
department         0         1         2         3
gender                                            
F           0.114035  0.203390  0.256098  0.254386
M           0.240741  0.233333  0.384615  0.290526

In [134]:
df.groupby('race')['decision'].mean()

race
B    0.247191
H    0.232759
W    0.252830
Name: decision, dtype: float64

In [135]:
df.groupby(['race','department']).mean().unstack()

decision                              
department         0         1         2         3
race                                              
B           0.166667  0.352941  0.222222  0.250000
H           0.200000  0.142857  0.344828  0.215686
W           0.148148  0.200000  0.250000  0.292829

In [116]:
# now detect and compare which trends
np.argmax(df.groupby('gender')['decision'].mean())

/home/smb/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


'M'

In [117]:
[gender_list[g] for g in np.argmax(df.groupby(['gender','department']).mean().unstack().values,axis=0)]

['M', 'F', 'F', 'M']